In [1]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import tensorflow as tf
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

In [2]:
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
import pandas as pd
import numpy as np
import os

# Path to the environment variables file .env
env_path = '/data/dssg-disinfo/.env'
load_dotenv(env_path, override=True)
PATH = os.getenv("PATH")
CLEAN_DATA = os.getenv("CLEAN_DATA")
df = pd.read_csv(os.path.join(PATH, CLEAN_DATA))

In [3]:
### Create sequence
vocabulary_size = 189492
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['article_text'])
sequences = tokenizer.texts_to_sequences(df['article_text'])
data = pad_sequences(sequences, maxlen=50)

In [4]:
# adding early stopping function
earlystop_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_accuracy', min_delta=0.1,
  patience=1)

In [5]:
# creating an embedding matrix

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [6]:
embedding_dim = 200
word_index = tokenizer.word_index
embedding_mat = create_embedding_matrix('../../../../../data/dssg-disinfo/coronavirus-corpus/chunks/wv.txt', word_index, embedding_dim)

In [7]:
## Network architecture
model = Sequential()
model.add(Embedding(189492, embedding_dim, 
                           weights=[embedding_mat], 
                           input_length=50, 
                           trainable=False))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model.fit(data, np.array(df['label']), validation_split=0.4, epochs=4, callbacks=[earlystop_callback])

Epoch 1/4
533/533 [==============================] - 103s 193ms/step - loss: 0.6041 - accuracy: 0.6629 - val_loss: 0.7897 - val_accuracy: 0.5388
Epoch 2/4
533/533 [==============================] - 108s 203ms/step - loss: 0.5848 - accuracy: 0.6918 - val_loss: 0.9373 - val_accuracy: 0.3991
